In [3]:
#import dataset
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import SCORERS
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler



df_original=pd.read_csv('MedalTable2.csv', sep=';')

#Initial Cleaning
df=df_original[(df_original["FlagY1"]==1) & (df_original["FlagY2"]==1) & (df_original["FlagY3"]==1)]
df=df.drop(["Silver", "Bronze", "Aux", "Country", "Year-1", "Year-2", "Year-3", "Year+1","FlagY1", "FlagY2"], axis=1)

#separating validation set (2016) and train and test datasets for total medal prediction
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler

df_validate=df_original[df_original["Year"]==2016][["Home", "TotalYear-1", "TotalYear-2", "TotalYear-3","HomeBefore", "HomeAfter", "TotalMedals", "Total"]]
df=df[df["Year"]<2016]
X=df[["Home", "TotalYear-1", "TotalYear-2","TotalYear-3","HomeBefore", "HomeAfter", "TotalMedals"]]
y=df["Total"]


In [4]:
#Ok, now we make Rio 2016 predictions, calculate R2 and check the top 20 (with Brazil) results

#Preparing x and y
X_linear=df[["Home", "TotalYear-1", "TotalYear-2", "TotalYear-3"]]
X_forest=df[["Home", "TotalYear-1", "TotalYear-2", "TotalMedals", "TotalYear-3"]]
X_svr=df[["Home", "TotalYear-1", "TotalYear-2", "TotalMedals", "TotalYear-3"]]
y_fit=df["Total"]

X_linear_validate=df_validate[["Home", "TotalYear-1", "TotalYear-2", "TotalYear-3"]]
X_forest_validate=df_validate[["Home", "TotalYear-1", "TotalYear-2", "TotalMedals", "TotalYear-3"]]
X_svr_validate=df_validate[["Home", "TotalYear-1", "TotalYear-2", "TotalMedals", "TotalYear-3"]]
y_validate=df_validate["Total"]

#normalizing
scaler = MinMaxScaler()
X_linear_scaled=scaler.fit_transform(X_linear)
X_linear_validate_scaled=scaler.transform(X_linear_validate)
X_forest_scaled=scaler.fit_transform(X_forest)
X_forest_validate_scaled=scaler.transform(X_forest_validate)
X_svr_scaled=scaler.fit_transform(X_svr)
X_svr_validate_scaled=scaler.transform(X_svr_validate)


#fitting, predicting and evaluating
linear=LinearRegression()
forest=RandomForestRegressor()
svr=SVR(C= 100, gamma='scale', kernel= 'rbf')

linear.fit(X_linear_scaled, y_fit)
forest.fit(X_forest_scaled, y_fit)
svr.fit(X_svr_scaled, y_fit)

y_linear=linear.predict(X_linear_validate_scaled)
y_forest=forest.predict(X_forest_validate_scaled)
y_svr=svr.predict(X_svr_validate_scaled)

df_validate["y_linear"]=y_linear
df_validate["y_forest"]=y_forest
df_validate["y_svr"]=y_svr
df_validate["Country"]=df_original[df_original["Year"]==2016]["Country"]

linear_score=r2_score(y_linear, y_validate)
forest_score=r2_score(y_forest, y_validate)
svr_score=r2_score(y_svr, y_validate)
#median_score=r2_score(df_validate["y_median"], y_validate)

print("Linear model r2:", linear_score, "\n Random Forest r2:", forest_score, "\n SVR r2:", svr_score, "\n Median r2:", )

#adding to df_validate and showing the top 20



df_validate.head(20)


Linear model r2: 0.8911067284629205 
 Random Forest r2: 0.9048779424170098 
 SVR r2: 0.9089071707220737 
 Median r2:


,Home,TotalYear-1,TotalYear-2,TotalYear-3,HomeBefore,HomeAfter,TotalMedals,Total,y_linear,y_forest,y_svr,Country
1161,0,104,112,101,0,0,973,121,98.097792,98.29,99.049765,USA
1162,0,65,51,30,1,0,973,67,48.930857,68.56,58.291222,GBR
1163,0,91,100,63,0,0,973,70,81.229401,80.25,85.424243,CHN
1164,0,68,60,90,0,0,973,56,65.569881,80.94,72.564455,RUS
1165,0,44,41,49,0,0,973,42,41.561723,36.63,43.784717,GER
1166,0,38,25,37,0,0,973,41,31.743276,27.36,35.180375,JPN
1167,0,35,43,33,0,0,973,42,35.221701,33.00,34.283008,FRA
1168,0,30,32,30,0,0,973,21,29.235250,26.02,28.728111,KOR
1169,0,28,27,32,0,0,973,28,27.333818,21.72,26.963896,ITA
1170,0,35,46,50,0,0,973,29,39.736966,35.08,37.275052,AUS
